In [1]:
# links:  https://stackoverflow.com/questions/30627968/merge-pandas-dataframes-where-one-value-is-between-two-others
#https://stackoverflow.com/questions/24614474/pandas-merge-on-name-and-closest-date

In [2]:
import pandas as pd
import sqlite3
from datetime import datetime

#We'll use firelynx's tables:
presidents = pd.DataFrame({"name": ["Bush", "Obama", "Trump"],
                           "president_id":[43, 44, 45]})
terms = pd.DataFrame({'start_date': pd.date_range('2001-01-20', periods=5, freq='48M'),
                      'end_date': pd.date_range('2005-01-21', periods=5, freq='48M'),
                      'president_id': [43, 43, 44, 44, 45]})
war_declarations = pd.DataFrame({"date": [datetime(2001, 9, 14), datetime(2003, 3, 3)],
                                 "name": ["War in Afghanistan", "Iraq War"]})
#Make the db in memory
conn = sqlite3.connect(':memory:')
#write the tables
terms.to_sql('terms', conn, index=False)
presidents.to_sql('presidents', conn, index=False)
war_declarations.to_sql('wars', conn, index=False)

qry = '''
    select  
        start_date PresTermStart,
        end_date PresTermEnd,
        wars.date WarStart,
        presidents.name Pres
    from
        terms join wars on
        date between start_date and end_date join presidents on
        terms.president_id = presidents.president_id
    '''
df = pd.read_sql_query(qry, conn)

In [3]:
presidents

,name,president_id
0,Bush,43
1,Obama,44
2,Trump,45


In [4]:
terms

,start_date,end_date,president_id
0,2001-01-31,2005-01-31,43
1,2005-01-31,2009-01-31,43
2,2009-01-31,2013-01-31,44
3,2013-01-31,2017-01-31,44
4,2017-01-31,2021-01-31,45


In [5]:
war_declarations

,date,name
0,2001-09-14,War in Afghanistan
1,2003-03-03,Iraq War


In [6]:
df

,PresTermStart,PresTermEnd,WarStart,Pres
0,2001-01-31 00:00:00,2005-01-31 00:00:00,2001-09-14 00:00:00,Bush
1,2001-01-31 00:00:00,2005-01-31 00:00:00,2003-03-03 00:00:00,Bush


In [7]:
cat = ['790036', '860081']

data_dir = '../Data/big_cats/Access_DB_table_exports/'
biochem = pd.read_excel(data_dir + 'Biochemistry ReportsTZ_ZV_WA_SA_MG_DM.xlsx')
biochem['Date'] = pd.to_datetime(biochem['Date'])
biochem = biochem[biochem['ARKS Number'].isin(cat)][['ARKS Number', 'Date', 'Creatinine (umol/L)']]
biochem['ARKS No'] = biochem['ARKS Number']
biochem.drop('ARKS Number', axis = 1, inplace=True)
# add an id column
biochem['biochem id'] = biochem.index

data_dir = '../Data/big_cats/Access_DB_table_exports/'
haem = pd.read_excel(data_dir + 'Haematology ReportsTZ_ZV_MG.xlsx')
haem['Date'] = pd.to_datetime(haem['Date'])
haem = haem[haem['ARKS No'].isin(cat)][['ARKS No', 'Date', 'RBC (x 10^12/L)']]
# add an id column
haem['haem id'] = haem.index

data_dir = '../Data/big_cats/Access_DB_table_exports/'
urine = pd.read_excel(data_dir + 'UrinalysisTZ_ZV_WA_SA_MG_DM.xlsx')
urine['Date of Sample Analysis'] = pd.to_datetime(urine['Date of Sample Analysis'])
urine = urine[urine['ARKS No'].isin(cat)][['ARKS No', 'Date of Sample Analysis', 'Urine Specific Gravity']]
urine['Date'] = urine['Date of Sample Analysis']
urine.drop('Date of Sample Analysis', axis = 1, inplace=True)
# add an id column
urine['urine id'] = urine.index

In [8]:
# do some im,portant stuff
haem = haem.sort_values(by=['Date'])
biochem = biochem.sort_values(by=['Date'])
urine = urine.sort_values(by=['Date'])

biochem['biochem date'] = biochem['Date']
haem['haem date'] = haem['Date']
urine['urine date'] = urine['Date']

# throw away duplicate dates
haem.drop_duplicates(subset=['ARKS No', 'Date'], keep='last', inplace=True)
biochem.drop_duplicates(subset=['ARKS No', 'Date'], keep='last', inplace=True)
urine.drop_duplicates(subset=['ARKS No', 'Date'], keep='last', inplace=True)

In [9]:
biochem

,Date,Creatinine (umol/L),ARKS No,biochem id,biochem date
2,1987-06-10,170.0,790036,2,1987-06-10
7,1992-04-03,208.0,860081,7,1992-04-03
3,1993-01-28,135.0,790036,3,1993-01-28
4,1993-05-10,143.0,790036,4,1993-05-10
8,1993-10-06,193.0,860081,8,1993-10-06
6,1996-07-18,91.8,790036,6,1996-07-18
9,2000-08-24,187.0,860081,9,2000-08-24
10,2003-03-19,263.0,860081,10,2003-03-19
11,2007-03-29,260.0,860081,11,2007-03-29


In [10]:
haem

,ARKS No,Date,RBC (x 10^12/L),haem id,haem date
97,790036,1987-06-10,NaN,97,1987-06-10
178,790036,1988-06-08,8.83,178,1988-06-08
266,790036,1989-04-07,NaN,266,1989-04-07
563,790036,1993-01-28,NaN,563,1993-01-28
562,790036,1993-05-10,NaN,562,1993-05-10
491,790036,1996-07-18,NaN,491,1996-07-18


In [11]:
urine

,ARKS No,Urine Specific Gravity,Date,urine id,urine date
11,790036,1.080,1989-08-01,11,1989-08-01
10,790036,NaN,1990-09-14,10,1990-09-14
9,790036,>1.050,1991-03-21,9,1991-03-21
8,790036,>1.050,1992-07-06,8,1992-07-06
7,790036,1.038,1992-07-13,7,1992-07-13
21,860081,1.072,1992-07-31,21,1992-07-31
6,790036,2.068,1993-01-28,6,1993-01-28
20,860081,1.068,1993-09-20,20,1993-09-20
15,860081,1.060,1993-09-24,15,1993-09-24
12,860081,1.074,1993-09-30,12,1993-09-30


In [12]:
# the magic happens below, check the direction and tolerance arguments
# if you want you can make a maximum tolerance on which to merge data

tol = pd.Timedelta('3 day')
#df3 = pd.merge_asof(left=df1,right=df2,right_index=True,left_index=True,direction='nearest',tolerance=tol)

df3 = pd.merge_asof(by=['ARKS No'], left=biochem,right=haem, left_on='Date', right_on='Date', direction='nearest',tolerance=tol)
df3

,Date,Creatinine (umol/L),ARKS No,biochem id,biochem date,RBC (x 10^12/L),haem id,haem date
0,1987-06-10,170.0,790036,2,1987-06-10,NaN,97.0,1987-06-10
1,1992-04-03,208.0,860081,7,1992-04-03,NaN,NaN,NaT
2,1993-01-28,135.0,790036,3,1993-01-28,NaN,563.0,1993-01-28
3,1993-05-10,143.0,790036,4,1993-05-10,NaN,562.0,1993-05-10
4,1993-10-06,193.0,860081,8,1993-10-06,NaN,NaN,NaT
5,1996-07-18,91.8,790036,6,1996-07-18,NaN,491.0,1996-07-18
6,2000-08-24,187.0,860081,9,2000-08-24,NaN,NaN,NaT
7,2003-03-19,263.0,860081,10,2003-03-19,NaN,NaN,NaT
8,2007-03-29,260.0,860081,11,2007-03-29,NaN,NaN,NaT


In [13]:
df4 = pd.merge_asof(by=['ARKS No'], left=haem,right=biochem, left_on='Date', right_on='Date', direction='nearest',tolerance=tol)
df4

,ARKS No,Date,RBC (x 10^12/L),haem id,haem date,Creatinine (umol/L),biochem id,biochem date
0,790036,1987-06-10,NaN,97,1987-06-10,170.0,2.0,1987-06-10
1,790036,1988-06-08,8.83,178,1988-06-08,NaN,NaN,NaT
2,790036,1989-04-07,NaN,266,1989-04-07,NaN,NaN,NaT
3,790036,1993-01-28,NaN,563,1993-01-28,135.0,3.0,1993-01-28
4,790036,1993-05-10,NaN,562,1993-05-10,143.0,4.0,1993-05-10
5,790036,1996-07-18,NaN,491,1996-07-18,91.8,6.0,1996-07-18


In [14]:
# nat in haem date  (matched on right col) mean that there was no match within tolerance
# need to decide what to do when a cat has 2 visits on the same day.  We can use id to filter this. Or deal with it in the original table.

In [15]:
df5 = pd.merge_asof(by=['ARKS No'], left=df4,right=urine, left_on='Date', right_on='Date', direction='nearest',tolerance=tol)
df5

,ARKS No,Date,RBC (x 10^12/L),haem id,haem date,Creatinine (umol/L),biochem id,biochem date,Urine Specific Gravity,urine id,urine date
0,790036,1987-06-10,NaN,97,1987-06-10,170.0,2.0,1987-06-10,NaN,NaN,NaT
1,790036,1988-06-08,8.83,178,1988-06-08,NaN,NaN,NaT,NaN,NaN,NaT
2,790036,1989-04-07,NaN,266,1989-04-07,NaN,NaN,NaT,NaN,NaN,NaT
3,790036,1993-01-28,NaN,563,1993-01-28,135.0,3.0,1993-01-28,2.068,6.0,1993-01-28
4,790036,1993-05-10,NaN,562,1993-05-10,143.0,4.0,1993-05-10,NaN,NaN,NaT
5,790036,1996-07-18,NaN,491,1996-07-18,91.8,6.0,1996-07-18,>1.050,4.0,1996-07-18


In [16]:
# take only triple matches
triples = df5.dropna(subset=['haem date', 'biochem date', 'urine date'], how='any')
triples

,ARKS No,Date,RBC (x 10^12/L),haem id,haem date,Creatinine (umol/L),biochem id,biochem date,Urine Specific Gravity,urine id,urine date
3,790036,1993-01-28,NaN,563,1993-01-28,135.0,3.0,1993-01-28,2.068,6.0,1993-01-28
5,790036,1996-07-18,NaN,491,1996-07-18,91.8,6.0,1996-07-18,>1.050,4.0,1996-07-18


In [19]:
triple_haem_ids = list(triples['haem id'])
triple_biochem_ids = list(triples['biochem id'])
triple_urine_ids = list(triples['urine id'])

# drop these ids from original data frames
haem_drop_triple = haem[~haem['haem id'].isin(triple_haem_ids)]
biochem_drop_triple = biochem[~biochem['biochem id'].isin(triple_biochem_ids)]
urine_drop_triple = urine[~urine['urine id'].isin(triple_urine_ids)]

In [20]:
# now find doubles!
df_hb = pd.merge_asof(by=['ARKS No'], left=haem_drop_triple,right=biochem_drop_triple, left_on='Date', right_on='Date', direction='nearest',tolerance=tol)
df_hb

,ARKS No,Date,RBC (x 10^12/L),haem id,haem date,Creatinine (umol/L),biochem id,biochem date
0,790036,1987-06-10,NaN,97,1987-06-10,170.0,2.0,1987-06-10
1,790036,1988-06-08,8.83,178,1988-06-08,NaN,NaN,NaT
2,790036,1989-04-07,NaN,266,1989-04-07,NaN,NaN,NaT
3,790036,1993-05-10,NaN,562,1993-05-10,143.0,4.0,1993-05-10


In [21]:
df_hu = pd.merge_asof(by=['ARKS No'], left=haem_drop_triple,right=urine_drop_triple, left_on='Date', right_on='Date', direction='nearest',tolerance=tol)
df_hu

,ARKS No,Date,RBC (x 10^12/L),haem id,haem date,Urine Specific Gravity,urine id,urine date
0,790036,1987-06-10,NaN,97,1987-06-10,NaN,NaN,NaT
1,790036,1988-06-08,8.83,178,1988-06-08,NaN,NaN,NaT
2,790036,1989-04-07,NaN,266,1989-04-07,NaN,NaN,NaT
3,790036,1993-05-10,NaN,562,1993-05-10,NaN,NaN,NaT


In [22]:
df_ub = pd.merge_asof(by=['ARKS No'], left=urine_drop_triple,right=biochem_drop_triple, left_on='Date', right_on='Date', direction='nearest',tolerance=tol)
df_ub

,ARKS No,Urine Specific Gravity,Date,urine id,urine date,Creatinine (umol/L),biochem id,biochem date
0,790036,1.080,1989-08-01,11,1989-08-01,NaN,NaN,NaT
1,790036,NaN,1990-09-14,10,1990-09-14,NaN,NaN,NaT
2,790036,>1.050,1991-03-21,9,1991-03-21,NaN,NaN,NaT
3,790036,>1.050,1992-07-06,8,1992-07-06,NaN,NaN,NaT
4,790036,1.038,1992-07-13,7,1992-07-13,NaN,NaN,NaT
5,860081,1.072,1992-07-31,21,1992-07-31,NaN,NaN,NaT
6,860081,1.068,1993-09-20,20,1993-09-20,NaN,NaN,NaT
7,860081,1.060,1993-09-24,15,1993-09-24,NaN,NaN,NaT
8,860081,1.074,1993-09-30,12,1993-09-30,NaN,NaN,NaT
9,860081,1.054,1993-10-01,16,1993-10-01,NaN,NaN,NaT


In [36]:
# take only double matches
double_hb = df_hb.dropna(subset=['haem date', 'biochem date'], how='any')
double_hu = df_hu.dropna(subset=['haem date', 'urine date'], how='any')
double_ub = df_ub.dropna(subset=['urine date', 'biochem date'], how='any')

# get ids for all combos
double_hb_h_ids = list(double_hb['haem id'])
double_hb_b_ids = list(double_hb['biochem id'])

double_hu_h_ids = list(double_hu['haem id'])
double_hu_u_ids = list(double_hu['urine id'])

double_ub_u_ids = list(double_ub['urine id'])
double_ub_b_ids = list(double_ub['biochem id'])

# drop these ids from data frames where triples have already been dropped
haem_singles = haem_drop_triple[~haem_drop_triple['haem id'].isin(double_hb_h_ids + double_hu_h_ids)]
biochem_singles = biochem_drop_triple[~biochem_drop_triple['biochem id'].isin(double_hb_b_ids + double_ub_b_ids)]
urine_singles = urine_drop_triple[~urine_drop_triple['urine id'].isin(double_hu_u_ids + double_ub_u_ids)]

In [37]:
# now concat everything
everything = pd.concat([triples, double_hb, double_hu, double_ub, haem_singles, biochem_singles, urine_singles], sort=False)

In [38]:
everything

,ARKS No,Date,RBC (x 10^12/L),haem id,haem date,Creatinine (umol/L),biochem id,biochem date,Urine Specific Gravity,urine id,urine date
3,790036,1993-01-28,NaN,563.0,1993-01-28,135.0,3.0,1993-01-28,2.068,6.0,1993-01-28
5,790036,1996-07-18,NaN,491.0,1996-07-18,91.8,6.0,1996-07-18,>1.050,4.0,1996-07-18
0,790036,1987-06-10,NaN,97.0,1987-06-10,170.0,2.0,1987-06-10,NaN,NaN,NaT
3,790036,1993-05-10,NaN,562.0,1993-05-10,143.0,4.0,1993-05-10,NaN,NaN,NaT
10,860081,1993-10-06,NaN,NaN,NaT,193.0,8.0,1993-10-06,1.040,14.0,1993-10-06
15,860081,2007-03-29,NaN,NaN,NaT,260.0,11.0,2007-03-29,1.030,17.0,2007-03-29
178,790036,1988-06-08,8.83,178.0,1988-06-08,NaN,NaN,NaT,NaN,NaN,NaT
266,790036,1989-04-07,NaN,266.0,1989-04-07,NaN,NaN,NaT,NaN,NaN,NaT
7,860081,1992-04-03,NaN,NaN,NaT,208.0,7.0,1992-04-03,NaN,NaN,NaT
9,860081,2000-08-24,NaN,NaN,NaT,187.0,9.0,2000-08-24,NaN,NaN,NaT


In [ ]:
# to do
# now we have a data frame with all the things we want!  Next do this for all big cat data